In [3]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'Tesla T4')

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/DLAA_2025"
%ls

Mounted at /content/drive
/content/drive/My Drive/DLAA_2025
check_training.ipynb  __pycache__/                        time_series.ipynb
dataset.jsonl         qwen2.5-0.5b-instruct-lora-output/  ts_check_training.py


In [5]:
!head dataset.jsonl

{"index": 0, "input": "\n/think Describe the time series in three sentences. First sentence: describe trend (increasing/decreasing/flat). Second sentence: noise intensity (low/medium/high). Third sentence: approximate localisation of global maximum (beginning/middle/end) and global minimum (beginning/middle/end).\nPut the description in a JSON format with the following pattern\n```json\n{ \"trend\": <sentence1>,\n  \"noise\": <sentence2>,\n  \"extrema\": <sentence3> }\n```\n Series: [00, 01, 00, 05, 07, 06, 09, 08, 10, 10, 12, 13, 16, 16, 17, 17, 15, 14, 14, 16, 18, 18, 19, 22, 26, 29, 26, 25, 24, 25, 24, 24, 26, 24, 24, 25, 27, 28, 26, 24, 26, 31, 30, 30, 30, 33, 34, 32, 33, 35, 35, 40, 41, 41, 44, 45, 45, 48, 48, 51, 49, 51, 53, 55, 54, 56, 59, 60, 57, 57, 58, 58, 60, 59, 60, 62, 61, 60, 60, 59, 59, 62, 65, 65, 63, 64, 63, 65, 68, 70, 74, 69, 71, 70, 70, 69, 70, 66, 69, 70, 69, 70, 72, 72, 74, 74, 74, 76, 76, 79, 79, 78, 79, 78, 80, 81, 82, 87, 90, 90, 92, 92, 90, 91, 92, 95, 95, 99]

In [6]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    TrainerCallback
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset, DatasetDict
import torch
import json
import os
import re
import logging

from sentence_transformers import SentenceTransformer, util

In [7]:
# === CONFIGURATION ===
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_PATH = "dataset.jsonl"

# Extract short model name
model_name_parts = MODEL_NAME.split('/')
model_short_name = model_name_parts[-1] if model_name_parts else "unknown_model"
OUTPUT_DIR = f"./{model_short_name.lower()}-lora-output"

print("Training outputs saved in:", OUTPUT_DIR)

Training outputs saved in: ./qwen2.5-0.5b-instruct-lora-output


In [8]:
# === LOAD JSONL DATASET ===
def load_jsonl_dataset(path):
    with open(path, 'r') as f:
        lines = [json.loads(l) for l in f]
    return Dataset.from_list(lines)

raw_dataset = load_jsonl_dataset(DATASET_PATH)
# only keep nb_samples first samples
nb_samples = 50
raw_dataset = raw_dataset.select(range(nb_samples)) ###########################################
split_dataset = raw_dataset.train_test_split(test_size=0.10)
dataset = DatasetDict({
    "train": split_dataset["train"],
    "test": split_dataset["test"]
})

print(f"Train size: {len(dataset['train'])}")
print(f"Test size: {len(dataset['test'])}")

Train size: 45
Test size: 5


In [9]:
# === TOKENIZER ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
####### xxxxx tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
# === PREPROCESSING ===
def tokenize(example):
    prompt = example["input"]
    response = example["output"]

    # Tokenize prompt and response separately
    prompt_ids = tokenizer(prompt, truncation=False)["input_ids"]
    response_ids = tokenizer(response, truncation=True, max_length=256)["input_ids"]

    # Calculate available space for the prompt
    max_total_len = 1024
    max_prompt_len = max_total_len - len(response_ids)
    prompt_ids = prompt_ids[-max_prompt_len:]  # Truncate if too long

    input_ids = prompt_ids + response_ids
    attention_mask = [1] * len(input_ids)

    # Labels: ignore prompt tokens
    labels = [-100] * len(prompt_ids) + response_ids

    # Padding if necessary
    pad_len = max_total_len - len(input_ids)
    if pad_len > 0:
        input_ids += [tokenizer.pad_token_id] * pad_len
        attention_mask += [0] * pad_len
        labels += [-100] * pad_len

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

print("Tokenizing data...")
tokenized_dataset = dataset.map(tokenize, remove_columns=dataset["train"].column_names)
print("Tokenization complete.")

Tokenizing data...


Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Tokenization complete.


In [11]:
# === MODEL ===
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True
)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [12]:
# === CONFIGURATION LORA ===
lora_config = LoraConfig(
    r=4,  # r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [13]:
# === SEMANTIC SIMILARITY MODEL ===
model_st = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")

# === SEMANTIC EVALUATION FUNCTION ===
def compute_semantic_similarity(model, tokenizer, dataset, output_file=None):
    model.eval()
    examples = dataset.to_list()
    inputs = [ex["input"] for ex in examples]
    gold_outputs = [ex["output"] for ex in examples]

    generated_outputs = []
    for inp in inputs:
        inputs_tokenized = tokenizer(inp, return_tensors="pt").to(model.device)
        prompt_len = inputs_tokenized.input_ids.shape[1]

        with torch.no_grad():
            output_ids = model.generate(
                **inputs_tokenized,
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7
            )

        generated_only_ids = output_ids[0][prompt_len:]
        output_text = tokenizer.decode(generated_only_ids, skip_special_tokens=True)
        generated_outputs.append(output_text)

    emb_generated = model_st.encode(generated_outputs, convert_to_tensor=True)
    emb_gold = model_st.encode(gold_outputs, convert_to_tensor=True)
    scores = torch.nn.functional.cosine_similarity(emb_generated, emb_gold)
    avg_score = float(scores.mean())

    if output_file:
        output_data = [{
            "input": inp,
            "generated_output": gen_out,
            "gold_output": gold_out,
            "score": float(score)
        } for inp, gen_out, gold_out, score in zip(inputs, generated_outputs, gold_outputs, scores)]

        with open(output_file, "w") as f:
            json.dump(output_data, f, indent=4)

    model.train()
    return avg_score

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
# === CALLBACK FOR EVALUATION AT EACH CHECKPOINT ===
class SemanticSimilarityCallback(TrainerCallback):
    def __init__(self, model, tokenizer, test_dataset, output_dir):
        self.model = model
        self.tokenizer = tokenizer
        self.test_dataset = test_dataset
        self.output_dir = output_dir

    def on_step_end(self, args, state, control, **kwargs):
        print(f"🔄 Step finished: {state.global_step}")

    def on_evaluate(self, args, state, control, **kwargs):
        print("\n✨ Evaluating semantic similarity at checkpoint save ✨\n")
        if not hasattr(self, "model") or self.model is None:
            print("Error: self.model is None.")
            return

        #trainer.model.eval()
        step = state.global_step
        print(f"creating evaluation file for checkpoint {step}")
        output_file = os.path.join(self.output_dir, f"evaluation_checkpoint-{step}.json")
        print(f"evaluation file: {output_file}")

        print(f"calculating semantic similarity for checkpoint {step}")
        score = compute_semantic_similarity(
            self.model,
            self.tokenizer,
            self.test_dataset,
            output_file=output_file
        )

        #trainer.model.train()
        print(f"\n✅ Checkpoint {step}: Semantic Similarity = {score:.4f}\n")

In [15]:
# === TRAINING ===
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    bf16=True,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=10,
    save_strategy="steps",
    eval_strategy="steps",
    eval_steps=20,
    save_steps=20,
    save_total_limit=10,
    report_to="none",
    max_steps=60,
    disable_tqdm=False
)

# Configure logging
logging.basicConfig(level=logging.INFO, force=True)

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    callbacks=[SemanticSimilarityCallback(model, tokenizer, dataset["test"], OUTPUT_DIR)],
)

# Add semantic evaluation callback
# semantic_callback = SemanticSimilarityCallback(tokenizer, dataset["test"], OUTPUT_DIR)
# trainer.add_callback(semantic_callback)

# Initial evaluation before training
print("\n🔍 Evaluating before training (semantic similarity)...")
score_before = compute_semantic_similarity(model, tokenizer, dataset["test"], output_file=os.path.join(OUTPUT_DIR, "evaluation_avant.json"))
print(f"Average score before training: {score_before:.4f}")

# Training
trainer.train()

/tmp/ipython-input-269036231.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



🔍 Evaluating before training (semantic similarity)...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Average score before training: 0.5963
🔄 Step finished: 1


...


✅ Checkpoint 40: Semantic Similarity = 0.9711

🔄 Step finished: 41
🔄 Step finished: 42
🔄 Step finished: 43
🔄 Step finished: 44
🔄 Step finished: 45
🔄 Step finished: 46
🔄 Step finished: 47
🔄 Step finished: 48
🔄 Step finished: 49
🔄 Step finished: 50
🔄 Step finished: 51
🔄 Step finished: 52
🔄 Step finished: 53
🔄 Step finished: 54
🔄 Step finished: 55
🔄 Step finished: 56
🔄 Step finished: 57
🔄 Step finished: 58
🔄 Step finished: 59
🔄 Step finished: 60

✨ Evaluating semantic similarity at checkpoint save ✨

creating evaluation file for checkpoint 60
evaluation file: ./qwen2.5-0.5b-instruct-lora-output/evaluation_checkpoint-60.json
calculating semantic similarity for checkpoint 60


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


✅ Checkpoint 60: Semantic Similarity = 0.9798



TrainOutput(global_step=60, training_loss=0.5045756061871847, metrics={'train_runtime': 563.1789, 'train_samples_per_second': 0.852, 'train_steps_per_second': 0.107, 'total_flos': 990264125030400.0, 'train_loss': 0.5045756061871847, 'epoch': 10.0})

In [16]:
%cd qwen2.5-0.5b-instruct-lora-output/
!pwd
!ls

/content/drive/MyDrive/DLAA_2025/qwen2.5-0.5b-instruct-lora-output
/content/drive/MyDrive/DLAA_2025/qwen2.5-0.5b-instruct-lora-output
checkpoint-20  evaluation_avant.json	      evaluation_checkpoint-60.json
checkpoint-40  evaluation_checkpoint-20.json  plotdiag
checkpoint-60  evaluation_checkpoint-40.json
